In [1]:
from TranscriptionDataset.ArrangementDataset import ArrangementDataModule
from TranscriptionModel.ArrangementModel import ArrangementModel
from pytorch_lightning import Trainer,seed_everything
import torch
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import os
from tqdm import tqdm

plt.rcParams['figure.figsize'] = [12, 8]
print(f"Cuda : {torch.cuda.is_available()}")
torch.set_float32_matmul_precision('medium')

seed_everything(42, workers=True)
# os.environ['WANDB_MODE'] = 'offline'

Global seed set to 42


Cuda : True


42

In [2]:
SAMPLE_RATE = 16000
dataset = "Trainsets/massive.hdf5"

In [3]:
data_module = ArrangementDataModule(
    location=dataset,
    sample_rate=SAMPLE_RATE,
    val_size=0.1,
    num_workers=10,
    batch_size=4,
    disableOhe=True
)

In [4]:
import json
import h5py
from TUtils import ArrangementUtils
import numpy as np
import tqdm

h5file = h5py.File(dataset, "r")
songsGroup = h5file["Songs"]
data = list(json.loads(songsGroup.attrs["index"]).values())
arrangements = np.array([0])
for item in tqdm.tqdm(data, desc="Iterating"):
    songGroup = h5file[f"/Songs/{item['group']}"]
    thisArrangement = [int(ArrangementUtils.arrangementIndex[arr]) + 3 for arr in songGroup.attrs["allArrangements"]]
    arrangement = np.zeros(6,dtype=bool)
    arrangement[thisArrangement] = True
#     print(arrangement,np.packbits(arrangement,bitorder="little"))
    arrangements = np.vstack((arrangements,np.packbits(arrangement,bitorder="little")))
arrangements = arrangements[1:]
ids,counts = np.unique(arrangements,return_counts=True)
total_number_of_classes = len(ids)
total_number_of_observations = np.sum(counts)
arrangementIndex = np.zeros(np.max(ids)+1)
class_weights = np.zeros(np.max(ids)+1)
ids,locations = np.unique(arrangements,return_counts=True)
for i in range(0,len(ids)):
    arrangementIndex[ids[i]] = counts[i]
    class_weights[ids[i]] = total_number_of_observations/(total_number_of_classes*counts[i])
# print(arrangementIndex.astype(int))
class_weights

Iterating: 100%|███████████████████████████████████████████████████████████████| 82766/82766 [00:53<00:00, 1544.39it/s]


array([0.00000000e+00, 3.66532629e-01, 0.00000000e+00, 3.25338050e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.21091270e+01,
       2.70406430e+00, 1.06886416e-01, 0.00000000e+00, 6.52893475e-01,
       0.00000000e+00, 1.72429167e+03, 0.00000000e+00, 1.50374273e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.39807432e+01, 1.00191265e+00, 0.00000000e+00, 1.96239568e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.89716667e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.72429167e+03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [5]:
model = ArrangementModel(
    arrangement_size=6,
    class_weight_index=class_weights
)

In [6]:
# train
wandb_logger = WandbLogger(
    project="ArrangementModelMSE"
)
trainer = Trainer(
    # profiler="simple",
    default_root_dir="model_weights/",
    accelerator="gpu",
    devices=-1,
    max_epochs=10,
    logger=wandb_logger
)

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name          | Type             | Params
----------------------------------------------------
0  | lossFunction  | Weighted_MSELoss | 0     
1  | conv1         | Conv1d           | 5.6 K 
2  | bn1           | BatchNorm1d      | 70    
3  | pool1         | MaxPool1d        | 0     
4  | conv2         | Conv1d           | 3.7 K 
5  | bn2           | BatchNorm1d      | 70    
6  | pool2         | MaxPool1d        | 0     
7  | conv3         | Conv1d           | 7.4 K 
8  | bn3           | BatchNorm1d      | 140   
9  | pool3         | MaxPool1d        | 0     
10 | conv4         | Conv1d           | 14.8 K
11 | bn4           | BatchNorm1d      | 140   
12 | pool4         | MaxPool1d        | 0     
13 | flattener     | Flatten          | 0     
14 | fcArrangement | Linear           | 16.3 M
15 | linear2       | Linear           | 100 K 
16 | linear3       | Linear           | 606   
17 | sigmoid       | Sigmoid          | 0     
-----------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

C:\Users\ritwi\anaconda3\envs\music\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.save_checkpoint(r"C:\Users\ritwi\Github\MusicTranscription\model_weights\ArrangementModel_Mini_60s_10epoch_over.ckpt")

In [ ]:
model = model.load_from_checkpoint(r"C:\Users\ritwi\Github\MusicTranscription\model_weights\ArrangementModel_Mini_60s_10epoch_over.ckpt")
model.eval()
data_module.setup("")
dataloader = data_module.val_dataloader()
expected = np.array([[0]*6])
model_output = np.array([[0]*6])
for batch in tqdm(enumerate(iter(dataloader))):
    section, tuning, arrangement = batch[1]
    output = model(section).squeeze(1).cpu().detach().numpy()
    arrangement = arrangement.cpu().detach().numpy()
    expected = np.vstack((expected, arrangement))
    model_output = np.vstack((model_output, output))

In [ ]:
print(expected.shape)
print(model_output.shape)
print(np.sum(expected[:,0]))
model_output[:,0]>0